# Imports

In [1]:
import os
import glob
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create Database

In [2]:
#Hier wird das Pattern für Zeitschriften festgelegt

PATTERN_FOR_FAZ = "**/*faz*.html"
PATTERN_FOR_SZ = "**/*sz*.html"
PATTERN_FOR_WELT = "**/*welt*.html"

In [3]:
all_path_to_faz = glob.glob(os.path.join(Path.cwd().parent / "input", PATTERN_FOR_FAZ))
all_path_to_sz = glob.glob(os.path.join(Path.cwd().parent / "input", PATTERN_FOR_SZ))
all_path_to_welt = glob.glob(os.path.join(Path.cwd().parent / "input", PATTERN_FOR_WELT))
all_path = all_path_to_faz + all_path_to_sz + all_path_to_welt

In [4]:
print(len(all_path_to_faz))
print(len(all_path_to_sz))
print(len(all_path_to_welt))
print(len(all_path))

92
92
92
276


## HTML Analysis for FAZ

In [5]:
all_path_to_faz[0]

'/Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/input/data-lake_2024_11/2024-11-23-faz.html'

In [6]:
with open(all_path_to_faz[0], "r") as f:
    html = f.read()

In [7]:
bs = BeautifulSoup(html, "html.parser")

In [8]:
teasers = bs.select(".teaser-object__title, .top1-teaser__body-title")

for teaser in teasers:
    # Titel extrahieren
    title = teaser.get_text(strip=True)

    # Zum nächsten Geschwisterelement gehen, um den passenden Text zu finden
    text_element = teaser.find_next(class_="teaser-object__teaser-text")
    text = text_element.get_text(strip=True) if text_element else " "

    print("Titel:", title)
    print("Text:", text)
    print("-" * 50)
print(len(teasers))

Titel: Der richtige Mann in der falschen Partei?
Text: Boris Pistorius verzichtet auf die Kandidatur und unterstützt Olaf Scholz. Doch der steht als gescheiterer Kanzler da. Und bis zum SPD-Parteitag im Januar kann noch viel passieren.
--------------------------------------------------
Titel: Das Ende einer bizarren K-Frage
Text: Pistorius hätte die Debatte im Keim ersticken können. Er ließ sie laufen. Gerade noch rechtzeitig zog er jetzt die Notbremse. Der Schaden für Scholz und die SPD ist dennoch groß.
--------------------------------------------------
Titel: Was das Northvolt-Debakel für Europa bedeutet
Text: Der schwedische Batterie-Pionier Northvolt kämpft ums Überleben. Kann Europa den Vorsprung der Konkurrenz aus Asien überhaupt noch wettmachen?
--------------------------------------------------
Titel: Das zähe Klimaringen in Baku nimmt kein Ende
Text: Auch die Verlängerung ins Wochenende hinein hat bisher keinen Durchbruch auf der Weltklimakonferenz gebracht. Sowohl Entwicklun

## HTML Analysis for SZ

In [9]:
all_path_to_sz[0]

'/Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/input/data-lake_2024_11/2024-11-05-sz.html'

In [10]:
with open(all_path_to_sz[0], "r") as f:
    html = f.read()

In [11]:
bs = BeautifulSoup(html, "html.parser")

In [12]:
articles = bs.find_all("article")

for article in articles:
    # Titel suchen
    title_element = article.select_one('[data-manual="teaser-title"]')
    title = title_element.get_text(strip=True) if title_element else " "

    # Text suchen
    text_element = article.select_one('[data-manual="teaser-text"]')
    text = text_element.get_text(strip=True) if text_element else " "

    print("Titel:", title)
    print("Text:", text)
    print("-" * 50)
print(len(articles))

Titel: Wehe, wenn Trump verliert
Text: Sollte Kamala Harris die US-Präsidentschaftswahl für sich entscheiden, dürfte ihr Kontrahent Himmel und Hölle in Bewegung setzen, um doch noch Präsident zu werden. Wie er das Ergebnis zu seinen Gunsten drehen könnte.
--------------------------------------------------
Titel: Das sagen die Umfragen - ein Überblick in Grafiken
Text:  
--------------------------------------------------
Titel: Auf diesen Sendern können Sie die US-Wahl live verfolgen
Text:  
--------------------------------------------------
Titel: Harris feiert Abschlusskundgebung mit US-Promis
Text: Die Demokratin setzt am letzten Wahlkampftag auf Superstars und gute Stimmung, ihr Konkurrent hetzt gegen Einwanderer und fantasiert über Käfigkämpfe. In einem Dorf im Norden ist die Wahl bereits gelaufen.
--------------------------------------------------
Titel: „Ich werde Donald Trump wählen, auch wenn ich ihn nicht leiden kann“
Text: Devon wählt republikanisch, Chloe demokratisch und El

## HTML Analysis for Welt

In [13]:
all_path_to_welt[0]

'/Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/input/data-lake_2024_11/2024-11-27-welt.html'

In [14]:
with open(all_path_to_welt[0], "r") as f:
    html = f.read()

In [15]:
bs = BeautifulSoup(html, "html.parser")

In [16]:
articles = bs.find_all("article")

for article in articles:
    # Titel suchen (Klasse .c-teaser__headline)
    title_element = article.select_one('.c-teaser__headline')
    title = title_element.get_text(strip=True) if title_element else " "
    
    # Text suchen (Klasse .c-teaser__intro)
    text_element = article.select_one('.c-teaser__intro')
    text = text_element.get_text(strip=True) if text_element else " "

    # Ausgabe der Ergebnisse
    print("Titel:", title)
    print("Text:", text)
    print("-" * 50)
print(len(articles))

Titel: Die Mammutpläne der Bahn – und die Realität
Text: 2025 soll die wichtige Strecke Hamburg-Berlin innerhalb von neun Monaten saniert werden. Die Ausgaben drohten deutlich teurer zu werden, als der ursprüngliche Plan vorsah. Nach dem Ampel-Aus ist zudem das Geld knapp. Doch wo wird gespart – und welche Folgen hat das?
--------------------------------------------------
Titel: SPD und BSW einigen sich auf Koalitionsvertrag in Brandenburg
Text:  
--------------------------------------------------
Titel: „Bedarf auf Jahre gedeckt“, sagt Kubicki über Grüne – Thüringens CDU-Chef fürchtet Kemmerich-Moment
Text:  
--------------------------------------------------
Titel: „Auf absolutem Tiefpunkt“ – Mit Pistorius stünde die SPD bei 21 Prozent, sagt der Forsa-Chef
Text:  
--------------------------------------------------
Titel: „Wir richten uns ein“ – 65 Prozent der Ukrainer wollen bleiben, sagt der Behördenchef
Text:  
--------------------------------------------------
Titel: 31-Jähriger i

## Create DataFrame

In [17]:
parts_of_path = all_path[0].split(os.sep)
path = os.path.join(parts_of_path[-2], parts_of_path[-1])

print(f"Filename: {path}")

Filename: data-lake_2024_11/2024-11-23-faz.html


In [18]:
def get_news_data(file_path):
    """Extrahiert die relevanten Daten aus einer HTML-Datei basierend auf dem Dateinamen."""
    with open(file_path, "r", encoding="utf-8") as f:
        html = f.read()
    
    bs = BeautifulSoup(html, "html.parser")
    filename = os.path.basename(file_path)
    date = "-".join(filename.split("-")[:3])  # Datum aus Dateiname extrahieren
    parts_of_path = file_path.split(os.sep)
    path = os.path.join(parts_of_path[-2], parts_of_path[-1])
    
    data = []
    
    if "faz" in filename:
        teasers = bs.select(".top1-teaser__body-title, .teaser-object__title")
        for teaser in teasers:
            title = teaser.get_text(strip=True)
            text_element = teaser.find_next(class_="teaser-object__teaser-text")
            text = text_element.get_text(strip=True) if text_element else " "
            data.append({"Zeitung": "FAZ", "Pfad": path, "Datum": date, "Überschrift": title, "Text": text})
    
    elif "sz" in filename:
        articles = bs.find_all("article")
        for article in articles:
            title_element = article.select_one('[data-manual="teaser-title"]')
            title = title_element.get_text(strip=True) if title_element else " "
            text_element = article.select_one('[data-manual="teaser-text"]')
            text = text_element.get_text(strip=True) if text_element else " "
            data.append({"Zeitung": "SZ", "Pfad": path, "Datum": date, "Überschrift": title, "Text": text})
    
    elif "welt" in filename:
        articles = bs.find_all("article")
        for article in articles:
            title_element = article.select_one('.c-teaser__headline')
            title = title_element.get_text(strip=True) if title_element else " "
            text_element = article.select_one('.c-teaser__intro')
            text = text_element.get_text(strip=True) if text_element else " "
            data.append({"Zeitung": "Welt", "Pfad": path, "Datum": date, "Überschrift": title, "Text": text})
    
    return data

In [19]:
output_dir = Path.cwd().parent / "output"
all_data = []

for file_path in all_path:
    all_data.extend(get_news_data(file_path))

# DataFrame erstellen
df = pd.DataFrame(all_data)

# SQLite-Datenbank speichern
db_path = output_dir / "news_data.sqlite"
conn = sqlite3.connect(db_path)
df.to_sql("news", conn, if_exists="replace", index=False)
conn.close()

print(f"Datenbank gespeichert unter: {db_path}")

Datenbank gespeichert unter: /Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/output/news_data.sqlite


# Create Result

In [20]:
path_to_sqlite = Path.cwd().parent / "output" / "news_data.sqlite"
path_to_sqlite

PosixPath('/Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/output/news_data.sqlite')

In [21]:
connection = sqlite3.connect(path_to_sqlite)
query = "SELECT * FROM news"
df = pd.read_sql_query(query, connection)
df.head()

,Zeitung,Pfad,Datum,Überschrift,Text
0,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,Der richtige Mann in der falschen Partei?,Boris Pistorius verzichtet auf die Kandidatur ...
1,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,Das Ende einer bizarren K-Frage,Pistorius hätte die Debatte im Keim ersticken ...
2,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,Was das Northvolt-Debakel für Europa bedeutet,Der schwedische Batterie-Pionier Northvolt käm...
3,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,Das zähe Klimaringen in Baku nimmt kein Ende,Auch die Verlängerung ins Wochenende hinein ha...
4,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,"„Wir kommen nicht so schnell voran, wie es mög...",Kommenden Montag findet in Berlin ein Long-Cov...


In [22]:
df.shape

(45143, 5)

## Data cleaning

In [23]:
df[(df["Überschrift"] == " ") & (df["Text"] != " ")].shape

(0, 5)

In [24]:
df[(df["Überschrift"] == " ") & (df["Text"] == " ")].shape

(557, 5)

In [25]:
45143-557

44586

In [26]:
df.drop(df[df["Überschrift"] == " "].index, inplace=True)
df.shape

(44586, 5)

### Lowercasing

In [27]:
df["Überschrift"] = df["Überschrift"].str.lower()
df["Text"] = df["Text"].str.lower()
df.head()

,Zeitung,Pfad,Datum,Überschrift,Text
0,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,der richtige mann in der falschen partei?,boris pistorius verzichtet auf die kandidatur ...
1,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,das ende einer bizarren k-frage,pistorius hätte die debatte im keim ersticken ...
2,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,was das northvolt-debakel für europa bedeutet,der schwedische batterie-pionier northvolt käm...
3,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,das zähe klimaringen in baku nimmt kein ende,auch die verlängerung ins wochenende hinein ha...
4,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,"„wir kommen nicht so schnell voran, wie es mög...",kommenden montag findet in berlin ein long-cov...


### Sonderzeichen

In [28]:
df["Überschrift"] = df["Überschrift"].str.replace(r'[^a-zA-ZäöüÄÖÜß -]', '', regex=True)
df["Text"] = df["Text"].str.replace(r'[^a-zA-ZäöüÄÖÜß -]', '', regex=True)
df.head()

,Zeitung,Pfad,Datum,Überschrift,Text
0,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,der richtige mann in der falschen partei,boris pistorius verzichtet auf die kandidatur ...
1,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,das ende einer bizarren k-frage,pistorius hätte die debatte im keim ersticken ...
2,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,was das northvolt-debakel für europa bedeutet,der schwedische batterie-pionier northvolt käm...
3,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,das zähe klimaringen in baku nimmt kein ende,auch die verlängerung ins wochenende hinein ha...
4,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,wir kommen nicht so schnell voran wie es mögli...,kommenden montag findet in berlin ein long-cov...


### Stopwords

In [ ]:
stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords = requests.get(stopwords_url, allow_redirects=True).text.split("\n")[9:] # hier cutten, weil davor keine Stopwords kommen
stopwords

['ab',
 'aber',
 'abermaliges',
 'abermals',
 'abgerufen',
 'abgerufene',
 'abgerufener',
 'abgerufenes',
 'abgesehen',
 'acht',
 'aehnlich',
 'aehnliche',
 'aehnlichem',
 'aehnlichen',
 'aehnlicher',
 'aehnliches',
 'aehnlichste',
 'aehnlichstem',
 'aehnlichsten',
 'aehnlichster',
 'aehnlichstes',
 'aeusserst',
 'aeusserste',
 'aeusserstem',
 'aeussersten',
 'aeusserster',
 'aeusserstes',
 'ähnlich',
 'ähnliche',
 'ähnlichem',
 'ähnlichen',
 'ähnlicher',
 'ähnliches',
 'ähnlichst',
 'ähnlichste',
 'ähnlichstem',
 'ähnlichsten',
 'ähnlichster',
 'ähnlichstes',
 'alle',
 'allein',
 'alleine',
 'allem',
 'allemal',
 'allen',
 'allenfalls',
 'allenthalben',
 'aller',
 'allerdings',
 'allerlei',
 'alles',
 'allesamt',
 'allg',
 'allg.',
 'allgemein',
 'allgemeine',
 'allgemeinem',
 'allgemeinen',
 'allgemeiner',
 'allgemeines',
 'allgemeinste',
 'allgemeinstem',
 'allgemeinsten',
 'allgemeinster',
 'allgemeinstes',
 'allmählich',
 'allzeit',
 'allzu',
 'als',
 'alsbald',
 'also',
 'am',
 '

In [30]:
def remove_stopwords(text, stopwords):
    # Den Text in Kleinbuchstaben umwandeln und dann in Wörter zerlegen
    words = text.lower().split()
    # Stopwords entfernen
    words = [word for word in words if word not in stopwords]
    # Die restlichen Wörter wieder zu einem Text zusammensetzen
    return " ".join(words)

In [31]:
df["Überschrift"] = df["Überschrift"].apply(lambda x: remove_stopwords(x, stopwords))
df["Text"] = df["Text"].apply(lambda x: remove_stopwords(x, stopwords))
df.head()

,Zeitung,Pfad,Datum,Überschrift,Text
0,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,richtige falschen partei,boris pistorius verzichtet kandidatur unterstü...
1,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,bizarren k-frage,pistorius debatte keim ersticken ließ laufen r...
2,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,northvolt-debakel europa bedeutet,schwedische batterie-pionier northvolt kämpft ...
3,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,zähe klimaringen baku,verlängerung wochenende durchbruch weltklimako...
4,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,,kommenden montag berlin long-covid-kongress ch...


## Create "text_combined"

In [32]:
df["text_combined"] = df["Überschrift"] + " " + df["Text"]
df.head()

,Zeitung,Pfad,Datum,Überschrift,Text,text_combined
0,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,richtige falschen partei,boris pistorius verzichtet kandidatur unterstü...,richtige falschen partei boris pistorius verzi...
1,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,bizarren k-frage,pistorius debatte keim ersticken ließ laufen r...,bizarren k-frage pistorius debatte keim erstic...
2,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,northvolt-debakel europa bedeutet,schwedische batterie-pionier northvolt kämpft ...,northvolt-debakel europa bedeutet schwedische ...
3,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,zähe klimaringen baku,verlängerung wochenende durchbruch weltklimako...,zähe klimaringen baku verlängerung wochenende ...
4,FAZ,data-lake_2024_11/2024-11-23-faz.html,2024-11-23,,kommenden montag berlin long-covid-kongress ch...,kommenden montag berlin long-covid-kongress c...


## Peform LDA

In [33]:
def perform_lda(df, n_topics=10):
    results = {}
    
    # Für jede Zeitung LDA ausführen
    for newspaper in df['Zeitung'].unique():
        # Filtere DataFrame nach Zeitung
        newspaper_df = df[df['Zeitung'] == newspaper]
        
        # CountVectorizer erstellen
        vectorizer = CountVectorizer(stop_words=stopwords)
        X = vectorizer.fit_transform(newspaper_df['text_combined'])
        
        # LDA Modell
        lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
        lda.fit(X)
        
        # Extrahiere die Top Wörter für jedes Thema
        feature_names = np.array(vectorizer.get_feature_names_out())
        topics = []
        
        for topic_idx, topic in enumerate(lda.components_):
            top_words_idx = topic.argsort()[:-11:-1]
            top_words = feature_names[top_words_idx]
            topics.append(" ".join(top_words))
        
        topic_distribution = lda.transform(X)
        # Durchschnitt über alle Dokumente
        avg_topic_distribution = topic_distribution.mean(axis=0)
        
        # Sortiere Topics nach Wahrscheinlichkeit & Indizes nach Wahrscheinlichkeit sortieren (absteigend)
        sorted_indices = np.argsort(avg_topic_distribution)[::-1]  # 
        
        sorted_topics = [topics[i] for i in sorted_indices]  # Sortierte Topics
        sorted_percentages = avg_topic_distribution[sorted_indices]  # Sortierte Wahrscheinlichkeiten
        sorted_percentages = (sorted_percentages / sorted_percentages.sum()) * 100  # In Prozent umrechnen
        
        # Ergebnisse speichern
        results[newspaper] = {"topics": sorted_topics, "percentages": sorted_percentages}
    
    return results

In [34]:
# LDA für das gesamte DataFrame durchführen
topics_by_newspaper = perform_lda(df)

# Ausgabe der Ergebnisse mit Prozenten
for newspaper, data in topics_by_newspaper.items():
    print(f"Top 10 Topics für {newspaper}:")
    for i, (topic, percentage) in enumerate(zip(data["topics"], data["percentages"]), 1):
        print(f"  Topic {i}: {topic} ({percentage:.2f}%)")
    print("\n")

/Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['dat', 'inf', 'zb'] not in stop_words.
  warnings.warn(


Top 10 Topics für FAZ:
  Topic 1: scholz merz assads spd afd beste syrien trump land grünen (11.26%)
  Topic 2: jahr welt deutschland blick faz besten deutschen daten frankfurt zeigen (10.72%)
  Topic 3: jahr reise jobs besten jahren vielfalt beste welt überblick bringt (10.53%)
  Topic 4: trump us trumps präsident donald usa liveblog geld deutschland präsidenten (10.36%)
  Topic 5: ki trump deutschland reise euro millionen deutsche lernen welt new (9.83%)
  Topic 6: deutschland deutsche stellt rolle spielt us grönland lässt bordeaux fokus (9.64%)
  Topic 7: menschen große beste reise kalifornien besten deutschland inklusive booklet jahre (9.63%)
  Topic 8: trump ki donald musk weihnachten elon faz beste deutschland bücher (9.62%)
  Topic 9: frankfurt hessen beste eintracht jobs eltern berlin leben lässt menschen (9.39%)
  Topic 10: deutschland deutschen faz menschen ukraine deutsche amazon etf funktioniert cdu (9.01%)


Top 10 Topics für SZ:
  Topic 1: wissen füllt felder is my trump 

## Plot Top Topics

In [35]:
def plot_top_topics(topics_by_newspaper, output_path):
    # Liste zur Speicherung der Daten für den Plot
    plot_data = []

    # Daten für jede Zeitung sammeln
    for newspaper, data in topics_by_newspaper.items():
        for i in range(3):  # Nur die Top 3 Topics nehmen
            plot_data.append({
                "Zeitung": newspaper,
                "Topic": f"{newspaper} - Topic {i+1}: {data['topics'][i]}",  # Zeitung + Topic Nummer
                "Wahrscheinlichkeit (%)": data["percentages"][i]
            })
    
    # DataFrame für die Visualisierung erstellen
    df_plot = pd.DataFrame(plot_data)

    # Stil setzen
    sns.set_theme(style="whitegrid")
    
    # Balkendiagramm mit Seaborn
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(
        data=df_plot, 
        x="Zeitung", 
        y="Wahrscheinlichkeit (%)", 
        hue="Topic",  # Hue enthält nun Zeitung + Topic
        palette="viridis"
    )
    
    # Titel & Labels
    plt.title("Top 3 Topics pro Zeitung", fontsize=14)
    plt.xlabel("Zeitung", fontsize=12)
    plt.ylabel("Wahrscheinlichkeit (%)", fontsize=12)
    plt.legend(title="Zeitung - Topic", bbox_to_anchor=(1, 1), loc='upper left')  # Legende mit Zeitung + Topic
    
    # Speichern als PDF
    plt.savefig(output_path, bbox_inches="tight")
    plt.close()  # Figure schließen, um Speicher freizugeben

In [37]:
output_path = Path.cwd().parent / "output"
plot_top_topics(topics_by_newspaper, output_path)

## Save Text Results

In [38]:
def save_text_results(topics_by_newspaper, output_path):
    with open(output_path, "w", encoding="utf-8") as file:
        for newspaper, data in topics_by_newspaper.items():
            file.write(f"Top 10 Topics für {newspaper}:\n")
            for i, (topic, percentage) in enumerate(zip(data["topics"], data["percentages"]), 1):
                file.write(f"  Topic {i}: {topic} ({percentage:.2f}%)\n")
            file.write("\n")  # Leerzeile für bessere Lesbarkeit

## Save all Results

In [39]:
pdf_path = os.path.join(Path.cwd().parent / "output", "result.pdf")
txt_path = os.path.join(Path.cwd().parent / "output", "full_result.txt")

In [40]:
plot_top_topics(topics_by_newspaper, pdf_path)
save_text_results(topics_by_newspaper, txt_path)

print(f"Grafik gespeichert unter: {pdf_path}")
print(f"Textdatei gespeichert unter: {txt_path}")

Grafik gespeichert unter: /Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/output/result.pdf
Textdatei gespeichert unter: /Users/antonioaleksic/Documents/ADSC1_Prüfungsleistung/ADSC1/output/full_result.txt
